
# Hyperparameter Optimization with Optuna

Tutorial

### Jefferson Fialho Coelho


In [5]:
import optuna
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
from optuna.trial import TrialState
from optuna.importance import MeanDecreaseImpurityImportanceEvaluator

## Optuna: Optimization Framework
    
### Paper:
[**Optuna: A Next-generation Hyperparameter Optimization Framework. In KDD.**](https://arxiv.org/abs/1907.10902)

Takuya Akiba, Shotaro Sano, Toshihiko Yanase, Takeru Ohta,and Masanori Koyama. 2019.

KDD '19: Proceedings of the 25th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining

### Overview of Optuna’s system design
<img src="img/optuna_flow.png"/>

## Optuna Key features

* Eager search spaces using bayesian methods with pruning system
* Efficiently search large spaces (Using prune unpromising trials for faster results)
  * Asynchronous Successive Halving (ASHA)
    * combine random search with principled early stopping in an asynchronous way
* Easy to parallelize hyperparameter searches over multiple threads/processes
  * We just need to run the python script passing the DB path

# How to install?

```
!pip install optuna
```

# Default pipeline

* Create a wrapper of your main algorithm to be optimized that receives an optuna object called `trial`

```
def objective(trial):
    ...
```

* Inside your `objective` function, use `optuna` methods to define your hiperparameters range:
   * `optuna.trial.Trial.suggest_categorical()` for categorical parameters
   * `optuna.trial.Trial.suggest_int()` for integer parameters
   * `optuna.trial.Trial.suggest_float()` for floating point parameters

E.g.:
```
# Categorical parameter
optimizer = trial.suggest_categorical("optimizer", ["MomentumSGD", "Adam"])

# Integer parameter
num_layers = trial.suggest_int("num_layers", 1, 3)

# Integer parameter (log)
num_channels = trial.suggest_int("num_channels", 32, 512, log=True)

# Integer parameter (discretized)
num_units = trial.suggest_int("num_units", 10, 100, step=5)

# Floating point parameter
dropout_rate = trial.suggest_float("dropout_rate", 0.0, 1.0)
```

* The `objective` function needs to return an evaluation parameter to be `maximized` or `minimized`
   * E.g.:
      * `maximization` of accuracy
      * `minimization` of EMD

* To provide data for `pruning` function works, we need to send an intermediate evaluation parameter value to `optuna` object

```
...
trial.report(accuracy, epoch)

if trial.should_prune():
    raise optuna.exceptions.TrialPruned()
...
```

* Create a `study` object to call the `object` function:
   * set the study name (the name of database table)
   * set a database to store the experiment data
   * set `load_if_exists=True` if you want to use a database that you already created 
   * set the optimization direction (`maximize` or `minimize`)

In [6]:
study = optuna.create_study(
    study_name='opt_tutorial',
    storage='sqlite:///tutorial.db',
    load_if_exists=True,
    direction="maximize")

[I 2022-04-06 13:46:57,711] Using an existing study with name 'opt_tutorial' instead of creating a new one.


* Run the optimization loop:

```
study.optimize(objective, n_trials=50,timeout=None)
```

* To check the values after (or while) optimization loop:

In [7]:
trial = study.best_trial

print("Best trial:")
print("    Value: ", trial.value)

print("Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
    Value:  0.828125
Params: 
    dropout_l0: 0.29733833743419186
    dropout_l1: 0.18955128033949867
    lr: 0.0015365378703487743
    n_layers: 2
    n_units_l0: 123
    n_units_l1: 114
    optimizer: RMSprop


## ok, but...
<img src="img/talk-is-cheap-show-me-the-code.jpg"/>

This tutorial will use the following notebooks to explain the implementation of the optimization study:

* `opt_tutorial_1.ipynb` (running model without optuna)
* `opt_tutorial_2.ipynb` (running the optimization study with optuna)
* `opt_tutorial_3.ipynb` (Data visualization)

# Tks!

**e-mail:** jefferson@fialhocoelho.com.br

**tutorial repo:** [github.com/fialhocoelho/optune-tutorial](https://github.com/fialhocoelho/optune-tutorial)